In [ ]:
import pickle
import pandas as pd
from gensim.summarization.textcleaner import clean_text_by_word
from gensim import downloader
import numpy as np
import re
import copy
import random

In [ ]:
def save_pkl(obj, pkl_name):
    with open(pkl_name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)
def load_pkl(pkl_name):
    with open(pkl_name, 'rb') as f:
        return pickle.load(f)  

In [ ]:
with open("../KGE/transe_wikidata5m.pkl", "rb") as fin:
    model = pickle.load(fin)
entity2id = model.graph.entity2id

entity_embeddings = model.solver.entity_embeddings

In [ ]:
def load_alias(alias_file):
    alias2object = {}
    ambiguous = set()
    with open(alias_file, "r") as fin:
        for line in fin:
            tokens = line.strip().split("\t")
            object = tokens[0]
            for alias in tokens[1:]:
                if alias in alias2object and alias2object[alias] != object:
                    ambiguous.add(alias)
                alias2object[alias] = object
        for alias in ambiguous:
            alias2object.pop(alias)
    return alias2object

In [ ]:
alias2entity = load_alias('/media/yuting/TOSHIBA EXT/KGE/entity.txt')

In [ ]:
wiki_entity = {}
for entity in alias2entity:
    try:
        wiki_entity[entity] = entity_embeddings[entity2id[alias2entity[entity]]]
    except:
        continue

In [ ]:
save_pkl(wiki_entity,"wiki_entity_en")

# test with one title

In [ ]:
title10w = pd.read_csv('../title_data/title10w.csv')

In [ ]:
title10w

In [ ]:
index2title = dict(zip(list(title10w.twe_index),list(title10w.twe_title)))

In [ ]:
# get dict1 index -> title words list dict2 index -> title entity
index2words = {}
index2entity = {}
for index in index2title.keys():
    
    # generate word list in title
    l_words = list(clean_text_by_word(index2title[index]).keys())
    if 'title' in l_words:
        l_words.remove('title')
    index2words[index] = l_words
    
    # map the entity in word list
    entity_list = []
    for i in l_words:
        try:
            entity = entity2id[alias2entity[str(i)]] # make sure it's a str
            entity_list.append(entity)
        except:
            continue
    index2entity[index] = entity_list

# load data

In [ ]:
# load the data
chunkSize = 200000
twe_text = pd.DataFrame()
chunk_df = pd.read_csv('/media/yuting/TOSHIBA EXT/retweet/retweetstext.csv', \
                       header=None, \
                       names=['id_retweet','text','date','user_re','user_orig','id_twe'],\
                       iterator=True, sep='#1#8#3#', \
                       chunksize=chunkSize)
for chunk in chunk_df:
    twe_text = twe_text.append(chunk)
    break

twe_text.replace(np.nan, 0, inplace=True)
twe_text.replace(np.inf, 0, inplace=True)

In [ ]:
# user statistics
user_re_list = list(dict(twe_text['user_re'].value_counts()).keys())
print(len(user_re_list))
user_orig_list = list(dict(twe_text['user_orig'].value_counts()).keys())
print(len(user_orig_list))
# common_user = [x for x in user_re_list if x in user_orig_list] #len(common_user) = 1296 #100w
# user_orig_not_re = [x for x in user_orig_list if x not in user_re_lsit] # len(user_orig_not_re) = 374598 # 100w
print(len(common_user))

In [ ]:
# dict user_index (index of all the tweets of chosed users)
users_re_index = dict()
for user in user_re_list:
    l_index = twe_text[twe_text['user_re'] == user].index.tolist()  # index of twe for users
    users_re_index[user] = l_index

In [ ]:
# dict user_index (index of all the tweets connect to news links)
users_re_index1 = dict()
for user in user_re_list:
    l_index1 = [x for x in users_re_index[user] if x in list(index2title.keys())]
    users_re_index1[user] = l_index1

# mapping user to words and entity

In [ ]:
# choose the retweet text with news links
twe_text_selected = twe_text.loc[list(title10w.twe_index)]

In [ ]:
twe_text_selected['title_words'] = twe_text_selected.index.map(lambda x: index2words[x])
twe_text_selected['title_entity'] = twe_text_selected.index.map(lambda x: index2entity[x])
twe_text_selected['twe_index'] = list(title10w['twe_index'])

In [ ]:
twe_text_selected[['id_retweet','user_re','user_orig','title_words','title_entity','twe_index']]

In [ ]:
# dict key:user_re, value:list of (user_orig, title_words, entities)
users_re_info = dict()
for user in user_re_list:
    user_orig_list = twe_text_selected[twe_text_selected['user_re'] == user].user_orig.tolist()
    words_list = twe_text_selected[twe_text_selected['user_re'] == user].title_words.tolist()
    entities_list = twe_text_selected[twe_text_selected['user_re'] == user].title_entity.tolist()
    users_re_info[user] = list(zip(user_orig_list,words_list,entities_list))

# node embedding & word embeddings & entitiy embedding

In [ ]:
# load word embedding
model_word = downloader.load("glove-twitter-100")

In [ ]:
# load node embedding
nodes_embedding = load_pkl('../weighted graph/sample_network_embedding_20w_100.pkl')

In [ ]:
# dict key: user_re, value:list of tuple (user_orig_emb, list of title_words_emb, list of entity_emb) 
users_info_embedding = {}
for key in users_re_info.keys():
    if len(users_re_info[key]) != 0:
        user_emb_node_title_entity = []
        for i in range(len(users_re_info[key])):
            orig_node = nodes_embedding[str(int(users_re_info[key][i][0]))]
            word_embed_list = []
            if len(users_re_info[key][i][1]) != 0:
                for j in range(len(users_re_info[key][i][1])):
                    try:
                        word_embed = model_word.word_vec(users_re_info[key][i][1][j])
                        word_embed_list.append(word_embed)
                    except:continue
            entity_emb_list = []
            if len(users_re_info[key][i][2]) != 0:
                for k in range(len(users_re_info[key][i][2])):
                    try:
                        entity_emb = entity_embeddings[users_re_info[key][i][2][k]]
                        entity_emb_list.append(entity_emb)
                    except:continue
            user_emb_node_title_entity.append((orig_node,word_embed_list,entity_emb_list))
    users_info_embedding[key] = user_emb_node_title_entity

In [ ]:
save_pkl(users_info_embedding, 'users_info_embedding_entity')

# entity statistics

In [ ]:
df_entity = pd.DataFrame(zip(list(index2entity.keys()),list(index2entity.values())), columns=['index','entities'])

In [ ]:
df_entity['entities_len'] = df_entity.entities.map(len)

In [ ]:
index2entity_len = dict(zip(list(df_entity.index),list(df_entity.entities_len)))

In [ ]:
import matplotlib.pyplot as plt
max_freq = max(index2entity_len.values())
print(max_freq)
bins = np.arange(0, 24,1)
plt.hist(index2entity_len.values(), bins, alpha=0.8, label='# history', color='blue')
plt.legend(loc='upper right')
plt.title('ditribution of user history number')

plt.xlabel('# history news')
plt.ylabel('frequency')
plt.show()

# mapping users to words

In [ ]:
index2title.keys()

In [ ]:
l_words = list(clean_text_by_word(index2title[13]).keys())
if 'title' in l_words:
        l_words.remove('title')
l_words

In [ ]:
l_emb = []
for i in range(len(l_words)):
    try:
        word_emb = model_word.word_vec(l_words[i])
        l_emb.append(word_emb)
    except:continue
l_emb

In [ ]:
index2title_words = {}
index2title_emb = {}
for index in index2title:
    l_words = list(clean_text_by_word(index2title[index]).keys())
    if 'title' in l_words:
        l_words.remove('title')
    index2title_words[index] = l_words
    
    l_words_emb = []
    for i in range(len(l_words)):
        try:
            word_emb = model_word.word_vec(l_words[i])
            l_words_emb.append(word_emb)
        except:continue
    index2title_emb[index] = l_words_emb

In [ ]:
# choose the retweet text with news links
twe_text_selected = twe_text.loc[list(title10w.twe_index)]
twe_text_selected

In [ ]:
twe_text_selected['title_words'] = twe_text_selected.index.map(lambda x: index2words[x])
twe_text_selected['twe_index'] = list(title10w['twe_index'])

In [ ]:
twe_text_selected = twe_text_selected[['id_retweet','user_re','user_orig','id_twe','title_words','twe_index']]

In [ ]:
index2orig = dict(zip(list(twe_text_selected.twe_index),list(twe_text_selected.user_orig)))

# choose data

In [ ]:
# history number dict
uid2titlesNum = dict(twe_text_selected['user_re'].value_counts())

In [ ]:
import matplotlib.pyplot as plt
max_freq = max(uid2titlesNum.values())
print(max_freq)
bins = np.arange(0, 80,1)
plt.hist(uid2titlesNum.values(), bins, alpha=0.8, label='# history', color='blue')
plt.legend(loc='upper right')
plt.title('ditribution of user history number')

plt.xlabel('# history news')
plt.ylabel('frequency')
plt.show()

In [ ]:
threshold = 1
uid_list = [k for k,v in uid2titlesNum.items() if v >= threshold]
len(uid_list)

In [ ]:
df = twe_text_selected[twe_text_selected.user_re.map(lambda x: x in uid_list)]

In [ ]:
df = df[(df.title_words.map(len)>1)]

In [ ]:
uid2index = df.groupby('user_re').twe_index.apply(list).to_dict()

# Ordering through time

In [ ]:
# in: dict key: user id, value: tuple(timestamp, user_orig, title)
# out: dict key: user id, value: tutple (user_orig, title) (arranged by time)
for user in dictHistory:
    l_new = dictHistory[user].sort(lambda x: x(0))
    dictHistory[user] = l_new

In [ ]:
df = pd.DataFrame.dictHistory(dictHistory, orient='index',columns=['history'])
df = df.reset_index().rename(columns = {'index':'id_retweet'})
df['user_orig'] = df.history.map(lambda x: x[0])
df['title_words'] = df.history.map(lambda x: x[1])

# negtive sampling

In [ ]:
num_neg_sample = 4 # train setup
num_neg_sample = 10 # test setup

In [ ]:
def negSampling(uid2titles, user, num_neg):
    
    samplingDict = {}
    samplingDict = copy.deepcopy(uid2titles)
    del samplingDict[user]  # remove the target user
    negPool = sum(list(samplingDict.values()),[])
    negSample = random.sample(negPool,num_neg)
    
    return negSample

In [ ]:
uid2neg = {}
for user in uid2index.keys():
    negSample = negSampling(uid2index, user, num_neg_sample)
    uid2neg[user] = negSample

# fill dataframe

In [ ]:
uid2pos = uid2index
l1 = list(uid2neg.keys())
l2 = list(uid2pos.keys())
assert l1 == l2

In [ ]:
df_pos = pd.DataFrame([(k,) + (ele,) + (1,) for k, v in uid2pos.items() for ele in v], columns=['uid', 'news','labels'])
df_neg = pd.DataFrame([(k,) + (ele,) + (0,) for k, v in uid2neg.items() for ele in v], columns=['uid', 'news','labels'])

df = pd.concat([df_pos,df_neg], axis=0)

history_num = 25
uid2news_fixed_len = {}
for user in uid2index.keys():
    indices = np.random.choice(list(range(0, len(uid2index[user]))), size=history_num, replace=True)
    chose_news = np.array(uid2index[user])[indices]
    uid2news_fixed_len[user] = chose_news

In [ ]:
df['history_titles'] = df['uid'].map(lambda x: uid2news_fixed_len[x])

In [ ]:
df['orig_uid'] = df.news.map(lambda x: index2orig[x])
df['history_orig_uid'] = df.history_titles.map(lambda x: [index2orig[y] for y in x])

# map node embeddings

In [ ]:
path = '/media/yuting/TOSHIBA EXT/retweet/diffusion/twitter/embeddings/infector_target3.txt'
with open(path, "r", encoding="utf-8") as f:
    lines = f.readlines()

In [ ]:
match_number = re.compile('-?\ *[0-9]+\.?[0-9]*(?:[Ee]\ *[-+]?\ *[0-9]+)?')
node2emb = {}
emb = ''
for i in range(len(lines)):
    
    if not lines[i].endswith(']\n'):
        emb = emb + lines[i]
        continue
    else:
        emb = emb + lines[i]
        l_temp = re.findall(match_number,emb)
        assert len(l_temp)==51
        number = [float(x) for x in l_temp[1:]]
        node2emb[l_temp[0]] =  number
        emb = ''
#     l_temp = re.findall(r"-?\d+\.?\d*e??\d*?",emb)   

In [ ]:
def mapNode(x,node2embed):
    user = str(int(x))
    try:
        emb = node2embed[user]
    except:
        emb = np.nan
    return emb

In [ ]:
def mapHistoryNode(x,node2embed):
    user = str(int(x))
    try:
        emb = node2embed[user]
    except:
        emb = [0] * 50
    return emb

In [ ]:
df['orig_node'] = df.orig_uid.apply(mapNode, args=(node2emb,))
df.dropna(inplace=True)
df['history_nodes'] = df.history_orig_uid.apply(lambda x: [mapHistoryNode(i,node2emb) for i in x])
df['news_emb'] = df.news.map(lambda x: index2title_emb[x])
df['history_news'] = df.history_titles.map(lambda x: [index2title_emb[x[i]] for i in range(len(x))])

In [ ]:
max_title_len = 10
embed_dim =100
nb_history = 20

In [ ]:
# extend title length
def titleLenExtend(title, max_title_len, embed_dim):
    if len(title) >= max_title_len:
        title = title[:max_title_len]
    else:
        m = int(max_title_len - len(title))
        addLen = [np.zeros(([embed_dim]),dtype=np.float32) for i in range(m)]
        title.extend(addLen)
    return title

In [ ]:
titles_test = [titleLenExtend(df.history_news.iloc[0][i],10,100) for i in range(nb_history)]
df.news_emb = df.news_emb.apply(titleLenExtend,args=(max_title_len,embed_dim))
df.history_news = df.history_news.apply(lambda x:[titleLenExtend(x[i],max_title_len,embed_dim) for i in range(len(x))])

In [ ]:
df['shape_hsitory_news'] = df.history_news.map(np.shape)
df['shape_news'] = df.news_emb.map(np.shape)

In [ ]:
save_pkl(df,'formed_data_history20_neg10_influence_emb_train')

In [ ]:
save_pkl(df,'formed_data_history20_neg10_influence_emb_test')